# Analisi di 22.5 valutazioni su Amazon

In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("AmazonReviews").setMaster("local")
sc = SparkContext.getOrCreate()

In [2]:
# Procuriamoci il Dataset
# Per prima cosa procuriamoci il dataset scaricandolo il locale, il dataset si trova in formato CSV a questo link, esegui il comando qui sotto per scaricarlo direttamente dal Notebook di DataBricks, che verrà salvato nella directory file:/databricks/driver.
# !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Books.csv

reviewsRDD = sc.textFile("./data/ratings_Books.csv")

# Recupero gli elementi

In [3]:
# Ogni elemento di ogni riga corrisponde a (in ordine):

#    Id dell'utente che ha lasciato la valutazione.
#    Id del libro recensito.
#    Valutazione da 1.0 a 5.0.
#    Timestamp di quando è stata lasciata la recensione.

# Per ovvi motivi di privacy non ci è possibile risalire ad un utente partendo dal suo ID, mentre per il libro 
# è possibile farlo aggiungendo l'ID a questo url https://www.amazon.com/dp/, ad esempio per il primo elemento: 
# https://www.amazon.com/dp/0000000116

# NOTA BENE Sì lo so, se hai cliccato sul link ti sei trovato una penna e non un libro come abbiamo detto, 
# il motivo è che tale penna è stata inserita impropriamente nella categoria libri di Amazon, quindi tutto 
# in regola per noi :).


reviewsRDD.take(5)

['AH2L9G3DQHHAJ,0000000116,4.0,1019865600',
 'A2IIIDRK3PRRZY,0000000116,1.0,1395619200',
 'A1TADCM7YWPQ8M,0000000868,4.0,1031702400',
 'AWGH7V0BDOJKB,0000013714,4.0,1383177600',
 'A3UTQPQPM4TQO0,0000013714,5.0,1374883200']

In [4]:
reviewsRDD = reviewsRDD.map(lambda x: x.split(","))
reviewsRDD.take(5)

[['AH2L9G3DQHHAJ', '0000000116', '4.0', '1019865600'],
 ['A2IIIDRK3PRRZY', '0000000116', '1.0', '1395619200'],
 ['A1TADCM7YWPQ8M', '0000000868', '4.0', '1031702400'],
 ['AWGH7V0BDOJKB', '0000013714', '4.0', '1383177600'],
 ['A3UTQPQPM4TQO0', '0000013714', '5.0', '1374883200']]

In [5]:
# Contiamo il numero totale di valutazioni
# Per contare il numero di recensioni usiamo semplicemente il metodo .count()

reviewsRDD.count()

22507155

# Quanti libri ci sono nel dataset?

In [6]:
booksRDD = reviewsRDD.map(lambda x: x[1])
booksRDD.take(5)

['0000000116', '0000000116', '0000000868', '0000013714', '0000013714']

In [7]:
uniqueRDD = booksRDD.distinct()
uniqueRDD.take(5)

['0001006657', '0001922408', '0002000601', '0002006650', '0002007770']

In [8]:
uniqueRDD.count()

2330066

In [14]:
#Oppure in un'unica riga
reviewsRDD.map(lambda x: x[1]).distinct().count()

2330066

   # Quante valutazioni ha ricevuto ogni libro?

In [9]:
booksCount = booksRDD.countByValue()

In [10]:
type(booksCount)

collections.defaultdict

In [12]:
i = 0
print("ID LIBRO\t CONTEGGIO")
for book_id, count in booksCount.items():
    print("%s\t%s" %(book_id, count))
    if(i>=10):
        break
    i+=1


ID LIBRO	 CONTEGGIO
0000000116	2
0000000868	1
0000013714	14
0000015393	1
0000029831	5
0000038504	2
0000041696	4
0000095699	1
0000174076	1
0000202010	1
0000230022	10


# Quali sono i 10 libri più valutati?

In [17]:
booksCount = booksRDD.map(lambda x: (x,1))
booksCount.take(5)

[('0000000116', 1),
 ('0000000116', 1),
 ('0000000868', 1),
 ('0000013714', 1),
 ('0000013714', 1)]

In [18]:
booksCount = booksCount.reduceByKey(lambda x,y: x+y)
booksCount.take(5)

[('0001006657', 2),
 ('0001922408', 2),
 ('0002000601', 6),
 ('0002006650', 2),
 ('0002007770', 6001)]

In [19]:
booksCountSorted = booksCount.sortBy(lambda x: x[1], ascending=False)
booksCountSorted.take(10)

[('0439023483', 21398),
 ('030758836X', 19867),
 ('0439023513', 14114),
 ('0385537859', 12973),
 ('0007444117', 12629),
 ('0375831002', 12571),
 ('038536315X', 12564),
 ('0345803485', 12290),
 ('0316055433', 11746),
 ('0849922070', 10424)]

In [21]:
booksRDD.map(lambda x: (x,1)) \
    .reduceByKey(lambda x,y: x+y) \
    .sortBy(lambda x: x[1], ascending=False) \
    .take(10)

[('0439023483', 21398),
 ('030758836X', 19867),
 ('0439023513', 14114),
 ('0385537859', 12973),
 ('0007444117', 12629),
 ('0375831002', 12571),
 ('038536315X', 12564),
 ('0345803485', 12290),
 ('0316055433', 11746),
 ('0849922070', 10424)]

# Qual è la valutazione media per ogni libro?

In [35]:
booksReviewRDD = reviewsRDD.map(lambda x: (x[1],float(x[2])))  
booksReviewRDD.take(5)

[('0000000116', 4.0),
 ('0000000116', 1.0),
 ('0000000868', 4.0),
 ('0000013714', 4.0),
 ('0000013714', 5.0)]

In [36]:
reviewSum = booksReviewRDD.reduceByKey(lambda x,y: x+y)
reviewSum.take(5)

[('0001006657', 10.0),
 ('0001922408', 10.0),
 ('0002000601', 23.0),
 ('0002006650', 8.0),
 ('0002007770', 26398.0)]

In [39]:
booksReviewRDD = booksReviewRDD.mapValues(lambda x: (x,1))
booksReviewRDD.take(5)

[('0000000116', (4.0, 1)),
 ('0000000116', (1.0, 1)),
 ('0000000868', (4.0, 1)),
 ('0000013714', (4.0, 1)),
 ('0000013714', (5.0, 1))]

In [40]:
reviewSum = booksReviewRDD.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
reviewSum.take(5)

[('0001006657', (10.0, 2)),
 ('0001922408', (10.0, 2)),
 ('0002000601', (23.0, 6)),
 ('0002006650', (8.0, 2)),
 ('0002007770', (26398.0, 6001))]

In [41]:
reviewMean = reviewSum.mapValues(lambda x: x[0]/x[1])
reviewMean.take(10)

[('0001006657', 5.0),
 ('0001922408', 5.0),
 ('0002000601', 3.8333333333333335),
 ('0002006650', 4.0),
 ('0002007770', 4.398933511081486),
 ('0002153327', 5.0),
 ('0002153904', 5.0),
 ('0002159260', 5.0),
 ('0002159627', 5.0),
 ('0002161621', 3.75)]

# Quali sono i 10 libri con la valutazione più alta?

In [44]:
#Considero solo i libri che hanno ricevuto almeno 100 recensioni

reviewMean = reviewSum.mapValues(lambda x: (x[0]/x[1], x[1]))
reviewMean.take(10)

[('0001006657', (5.0, 2)),
 ('0001922408', (5.0, 2)),
 ('0002000601', (3.8333333333333335, 6)),
 ('0002006650', (4.0, 2)),
 ('0002007770', (4.398933511081486, 6001)),
 ('0002153327', (5.0, 1)),
 ('0002153904', (5.0, 3)),
 ('0002159260', (5.0, 1)),
 ('0002159627', (5.0, 5)),
 ('0002161621', (3.75, 4))]

In [45]:
reviewMean = reviewMean.filter(lambda x: x[1][1]>=100)
reviewMean.count()

29296

In [47]:
reviewSorted = reviewMean.sortBy(lambda x: x[1][0], ascending=False)
reviewSorted.take(10)

[('0983408904', (5.0, 128)),
 ('0830766316', (5.0, 103)),
 ('0972394648', (4.992647058823529, 136)),
 ('1499390165', (4.991803278688525, 122)),
 ('0849381185', (4.990566037735849, 106)),
 ('0757317723', (4.9862068965517246, 145)),
 ('1939629071', (4.983193277310924, 119)),
 ('1499381921', (4.982857142857143, 350)),
 ('1616387165', (4.981308411214953, 107)),
 ('0814416993', (4.980769230769231, 104))]

In [49]:
# Faccio tutto in un'unica istruzione

reviewSum.filter(lambda x: x[1][1]>=100) \
    .mapValues(lambda x: (x[0]/x[1], x[1])) \
    .sortBy(lambda x: x[1][0], ascending=False) \
    .take(10)

[('0983408904', (5.0, 128)),
 ('0830766316', (5.0, 103)),
 ('0972394648', (4.992647058823529, 136)),
 ('1499390165', (4.991803278688525, 122)),
 ('0849381185', (4.990566037735849, 106)),
 ('0757317723', (4.9862068965517246, 145)),
 ('1939629071', (4.983193277310924, 119)),
 ('1499381921', (4.982857142857143, 350)),
 ('1616387165', (4.981308411214953, 107)),
 ('0814416993', (4.980769230769231, 104))]

# Chi sono i 10 recensori più critici?

In [50]:
reviewerRDD = reviewsRDD.map(lambda x: (x[0], (float(x[2]), 1)))
reviewerRDD.take(5)

[('AH2L9G3DQHHAJ', (4.0, 1)),
 ('A2IIIDRK3PRRZY', (1.0, 1)),
 ('A1TADCM7YWPQ8M', (4.0, 1)),
 ('AWGH7V0BDOJKB', (4.0, 1)),
 ('A3UTQPQPM4TQO0', (5.0, 1))]

In [51]:
reviewerRDD = reviewerRDD.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
reviewerRDD.take(5)

[('A2742OG8PK8KU6', (10.0, 2)),
 ('A2GKR2Q7MD8DG4', (12.0, 3)),
 ('A1MC4E00RO5E9T', (17.0, 4)),
 ('A3IKTM9D8RVWKU', (5.0, 1)),
 ('A3UZSIDE90JWW1', (5.0, 1))]

In [53]:
reviewerRDD = reviewerRDD.filter(lambda x: x[1][1]>100)
reviewerRDD.count()

11244

In [54]:
badReviewerRDD = reviewerRDD.mapValues(lambda x: x[0]/x[1])
badReviewerRDD.take(5)

[('A8IPQ1Q1O7YX5', 4.227048371174728),
 ('A2PN65B6BSTIYZ', 3.953271028037383),
 ('AX724J32HPG1J', 4.184738955823293),
 ('AFFGYGNO989PD', 4.2785714285714285),
 ('A1WCJEZS66D224', 3.5789473684210527)]

In [55]:
badReviewerSortedRDD = badReviewerRDD.sortBy(lambda x: x[1])
badReviewerSortedRDD.take(10)

[('AH62BQTCMR3BR', 1.0534188034188035),
 ('A186OSXC7LHJDB', 1.2014925373134329),
 ('A2HESNQJZ9OB7H', 1.2543859649122806),
 ('A36IQRD3B5MK8G', 1.505050505050505),
 ('A3JF63XRSLLH0P', 1.5648148148148149),
 ('A344N0X5LIV43M', 1.646551724137931),
 ('A1SS16UHYW77D4', 1.855421686746988),
 ('A19UFCMSFGOZ2K', 2.076923076923077),
 ('A1NJHOGKZZRAX8', 2.1588785046728973),
 ('A1ZY08GYVIKZFM', 2.2446043165467624)]

In [56]:
# Faccio tutto in un'unica istruzione

reviewsRDD.map(lambda x: (x[0], (float(x[2]), 1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
    .filter(lambda x: x[1][1]>100) \
    .mapValues(lambda x: x[0]/x[1]) \
    .sortBy(lambda x: x[1]) \
    .take(10)

[('AH62BQTCMR3BR', 1.0534188034188035),
 ('A186OSXC7LHJDB', 1.2014925373134329),
 ('A2HESNQJZ9OB7H', 1.2543859649122806),
 ('A36IQRD3B5MK8G', 1.505050505050505),
 ('A3JF63XRSLLH0P', 1.5648148148148149),
 ('A344N0X5LIV43M', 1.646551724137931),
 ('A1SS16UHYW77D4', 1.855421686746988),
 ('A19UFCMSFGOZ2K', 2.076923076923077),
 ('A1NJHOGKZZRAX8', 2.1588785046728973),
 ('A1ZY08GYVIKZFM', 2.2446043165467624)]